In [1]:
import time
import paho.mqtt.client as mqtt
import numpy as np
from collections import deque
from binascii import unhexlify,hexlify
from json import dumps
from calc_spo2 import calc_SPO2
from statistics import mean, stdev
from getmac import get_mac_address as gma
from serial_spo2 import *
from bloodPressure_rbd7k import *
# from thermalgun import *
from json import dumps, loads
from threading import Thread

from getComportclass import *
from serial_spo2 import *
rbp7k=readRBD7k()
#rbp7k.getSerObj()
# thermalgun=thermalgun()
spo2=readSPO2()
#spo2.getSerObj()

In [2]:
class mqttClient(mqtt.Client):
    def __init__(self):
        super().__init__()
        self.url_broker='3.0.54.110'
        self.port_broker=8082
        self.usr="mqtt"
        self.password="12345"
        self.publish_topic='vitalSign'
        self.subscribe_topic='remoteCtrl'
        self.devices={
#             "KRkristaz":thermalgun.sendCommand,             
            "Raycom_RPD7000": rbp7k.sendCommand,
            "spo2": spo2.sendCommand
                        }
        self.VitalSign={
#              "patientId":"None",
             "DID":gma(),
             "name":"Device Brand",
            #  "spo2":{"value":None,"raw":[],"hr":None},
            #  "hr":None,
            #  "temp":None,
            #  "pressure":{"dia":None,"sys":None,"hr":None},
             "dt": None
            }
        
    def on_connect(self,client, userdata, flags, rc):
        print("Connected with result code {0}".format(str(rc)))  # Print result of connection attempt
        client.subscribe(self.subscribe_topic)
        #client.subscribe(self.subscribe_stream)
    '''    
    def on_message(self,client, userdata, msg):
        print("Message received-> " + msg.topic + "=> " + str(msg.payload))
        message = loads(msg.payload.decode('utf8').replace("'", '"'))
        vitalSign=dict(self.VitalSign)
        if msg.topic == self.subscribe_topic:
            if message["NAME"] in self.devices:
                commandThread=Thread(target=self.devices[message["NAME"]],args=[message["CMD"]])
                commandThread.start()
                # vitalSign.update(self.devices[message["NAME"]](message["CMD"]))
        # self.publish(self.publish_topic,dumps(vitalSign))
    '''
    def on_message(self,client, userdata, msg):
        print("Message received-> " + msg.topic + ": " + str(msg.payload))
        message = loads(msg.payload.decode('utf8').replace("'", '"'))
        asyncio.run(self.devices[message["name"]](message["cmd"]))
        # actionThread=Thread(target=self.devices[message["name"]],args=(message["cmd"]))
        # actionThread.start()
        
    def on_publish(self, mqttc, obj, mid):
        print("mid: "+str(mid))

    def on_subscribe(self, mqttc, obj, mid, granted_qos):
        print("Subscribed: "+str(mid)+" "+str(granted_qos))

    def on_log(self, mqttc, obj, level, string):
        print(string)
        
    def run(self):
        self.username_pw_set(username=self.usr, password=self.password)
        self.connect(self.url_broker, self.port_broker, 60)
        
        rc = 0
        while rc == 0:
            rc = self.loop()
        if rc!=0:
            self.run()
        return rc

In [3]:
mqttc = mqttClient()
rc = mqttc.run()

print("rc: "+str(rc))

Sending CONNECT (u1, p1, wr0, wq0, wf0, c1, k60) client_id=b''
Received CONNACK (0, 0)
Connected with result code 0
Sending SUBSCRIBE (d0, m1) [(b'remoteCtrl', 0)]
Received SUBACK
Subscribed: 1 (0,)
Received PUBLISH (d0, q0, r0, m0), 'remoteCtrl', ...  (39 bytes)
Message received-> remoteCtrl: b'{"name":"Raycom_RPD7000","cmd":"start"}'
Caught exception in on_message: asyncio.run() cannot be called from a running event loop


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
rbp7k.ser.close()
spo2.ser.close()